In [1]:
# to read exposure cat from FITS
#version with the creation of lists, 5 x faster than with simuCat records
# as of 2016-10-11 18:44:48

import matplotlib
import matplotlib.pylab as plt

In [2]:
from lsst.meas.astrom import astrometry
import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.cameraGeom as cg
import lsst.afw.table as afwTable
import lsst.afw.image as afwImage
import numpy as np
import math
import cPickle as pickle
from astropy.io import fits
import random
import datetime
import starobs as so

In [3]:
# To simulate a square scan over a 3d x 3d field, each exp. 1d x 1d, displaced by 15' (1/4 d)
# We get in total 64 exp., we write a FITS catalog with coord(RA, dec), mag
def magfromflux(flux,ZP):
    return -2.5*math.log(flux, 10.) + ZP
def fluxfrommag(mag,ZP):
    return math.pow(10., -0.4*(mag - ZP))
def emagfromflux(flux,eflux):
    return 2.5*(eflux/flux)/math.log(10.)

itime = datetime.datetime.now()
raLim = [163., 166.71]; decLim = [34.5, 37.5];# Limits of field read in SDSS database
fieldSize = [raLim[1] - raLim[0], decLim[1] - decLim[0]]
expSize = [fieldSize[0]/3., fieldSize[1]/3.]
scan = [expSize[0]/4., expSize[1]/4.]
print fieldSize, scan
# name of exposure catalogue     
sra = '%04d' % (raLim[0]*10.)
sdec = '%04d' % (decLim[0]*10.)
# to read SDSS catalog
# name of SDSS catalogue     
#catName = 'sdss17' + sra + '_' + sdec + '.fit' # short catalog for tests 
catName = 'sdss' + sra + '_' + sdec + '.fit' 
sdssCat = afwTable.SourceCatalog.readFits(catName)
nStar = len(sdssCat); print 'nStar = ', nStar
random.seed(0) # to get always the same result
NPE17 = 194863.
NPEsky = 20200.
ZP = 17. + 2.5*math.log(NPE17, 10.)
lc = [[] for x in xrange(nStar)]
# to put indexes on exposure catalogue names 
# The model is 1 night = 8 exposures at a given dec, 8 different RA and xmass 
# fluxes are in PE
x = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8]
# a different k each night
k = [0.1, 0.12, 0.11, 0.14, 0.09, 0.08, 0.1, 0.07]
#ik = [0,1,2,3,4,5,6,7]
ik = [0,0,1,1,2,2,3,3]
# 2 different a for 5 and 3 nights
a = [-0.1, -0.25]
ia = [0,0,0,0,0,1,1,1]
nNight = len(k); nExp = len(x)

[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395


In [9]:
# !! the non parallel cell !! SKIP if parallel test

for idec in xrange(nNight) :
    for ira in xrange(nExp) :
# Catalogue file name
        catName = 'sdss' + sra + '_' + sdec + '_' + '%03d' % ira + '_' + '%03d' % idec
        expCat = afwTable.SourceCatalog.readFits(catName)
# Here match with sdssCat to get a unique star nr
        match = afwTable.matchRaDec(sdssCat, expCat, afwGeom.Angle(1./3600., afwGeom.degrees))
# Then loop over the match to fill light curve (lc)
        imax = 0
        for m in match :
            if imax>10:
                break
            imax = imax+1        
            istar  = m.first.get('id') - 1
#            record = simuCat.makeRecord()
            flux_r0 = m.second.get('flux_r')
            errFlux_r0 = m.second.get('errFlux_r')
# calculate the simulated signal and noise => mag and errMag
            u = magfromflux(flux_r0, ZP) - a[ia[idec]] + k[ik[idec]]*x[ira]
# error must be calculated for the effective nb PE count
            flux_r = fluxfrommag(u, ZP)
            errFlux_r = math.sqrt(flux_r + NPEsky)# test to reduce error bar
            flux_r = flux_r + random.gauss(0., errFlux_r)
# recalculate errFlux_r here !!!
            errFlux_r = math.sqrt(flux_r + NPEsky)# test to reduce error bar
            u = magfromflux(flux_r, ZP)
            eu = emagfromflux(flux_r, errFlux_r)
            obs = so.StarObs(math.pow(eu, -2.), u, idec, ira)
            lc[istar].append(obs)
    print catName, len(match), datetime.datetime.now()

sdss1630_0345_007_000 884 2016-10-13 17:43:45.157462
sdss1630_0345_007_001 803 2016-10-13 17:43:45.417935
sdss1630_0345_007_002 761 2016-10-13 17:43:45.616358
sdss1630_0345_007_003 780 2016-10-13 17:43:45.819877
sdss1630_0345_007_004 788 2016-10-13 17:43:46.019280
sdss1630_0345_007_005 791 2016-10-13 17:43:46.219672
sdss1630_0345_007_006 822 2016-10-13 17:43:46.419736
sdss1630_0345_007_007 853 2016-10-13 17:43:46.620236


In [4]:
# ! start here for parallel processing !

random.seed(0) # to get always the same result

def oneexp((ira,idec)):
    import lsst.afw.table as afwTable
    import lsst.afw.geom as afwGeom
    import lsst.afw.coord as afwCoord
    import numpy as np
    import math
    import cPickle as pickle
    from astropy.io import fits
    import random
    import datetime
    import starobs as so
    def magfromflux(flux,ZP):
        return -2.5*math.log(flux, 10.) + ZP
    def fluxfrommag(mag,ZP):
        return math.pow(10., -0.4*(mag - ZP))
    def emagfromflux(flux,eflux):
        return 2.5*(eflux/flux)/math.log(10.)
    itime = datetime.datetime.now()
    raLim = [163., 166.71]; decLim = [34.5, 37.5];# Limits of field read in SDSS database
    fieldSize = [raLim[1] - raLim[0], decLim[1] - decLim[0]]
    expSize = [fieldSize[0]/3., fieldSize[1]/3.]
    scan = [expSize[0]/4., expSize[1]/4.]
    print fieldSize, scan
# name of exposure catalogue     
    sra = '%04d' % (raLim[0]*10.)
    sdec = '%04d' % (decLim[0]*10.)
# to read SDSS catalog
# name of SDSS catalogue     
#catName = 'sdss17' + sra + '_' + sdec + '.fit' # short catalog for tests 
    pathname = '/renoir/fouchez/PROJECT/lsst/DM/notebooks/fabrice/notebooks/'
    catName = pathname + 'sdss' + sra + '_' + sdec + '.fit' 
    sdssCat = afwTable.SourceCatalog.readFits(catName)
    nStar = len(sdssCat); print 'nStar = ', nStar
    #random.seed(0) # to get always the same result
    NPE17 = 194863.
    NPEsky = 20200.
    ZP = 17. + 2.5*math.log(NPE17, 10.)
    lc = [[] for x in xrange(nStar)]
# to put indexes on exposure catalogue names 
# The model is 1 night = 8 exposures at a given dec, 8 different RA and xmass 
# fluxes are in PE
    x = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8]
# a different k each night
    k = [0.1, 0.12, 0.11, 0.14, 0.09, 0.08, 0.1, 0.07]
#ik = [0,1,2,3,4,5,6,7]
    ik = [0,0,1,1,2,2,3,3]
# 2 different a for 5 and 3 nights
    a = [-0.1, -0.25]
    ia = [0,0,0,0,0,1,1,1]
    nNight = len(k); nExp = len(x)
# Catalogue file name
    catName = pathname + 'sdss' + sra + '_' + sdec + '_' + '%03d' % ira + '_' + '%03d' % idec
    expCat = afwTable.SourceCatalog.readFits(catName)
# Here match with sdssCat to get a unique star nr
    match = afwTable.matchRaDec(sdssCat, expCat, afwGeom.Angle(1./3600., afwGeom.degrees))
# Then loop over the match to fill light curve (lc)
    for m in match :
        istar  = m.first.get('id') - 1
#            record = simuCat.makeRecord()
        flux_r0 = m.second.get('flux_r')
        errFlux_r0 = m.second.get('errFlux_r')
# calculate the simulated signal and noise => mag and errMag
        u = magfromflux(flux_r0, ZP) - a[ia[idec]] + k[ik[idec]]*x[ira]
# error must be calculated for the effective nb PE count
        flux_r = fluxfrommag(u, ZP)
        errFlux_r = math.sqrt(flux_r + NPEsky)# test to reduce error bar
        flux_r = flux_r + random.gauss(0., errFlux_r)
# recalculate errFlux_r here !!!
        errFlux_r = math.sqrt(flux_r + NPEsky)# test to reduce error bar
        u = magfromflux(flux_r, ZP)
        eu = emagfromflux(flux_r, errFlux_r)
        obs = so.StarObs(math.pow(eu, -2.), u, idec, ira)
        lc[istar].append(obs)
#    print catName, len(match), datetime.datetime.now() 
    return lc
    
#nNight = 8; nExp = 8
expo=[]
for idec in xrange(nNight) :
    for ira in xrange(nExp) :
        expo.append((idec,ira))
        
print expo        


[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)]


In [5]:
import ipyparallel as ipp
c = ipp.Client()
v = c[:]
c.ids


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [7]:
llc =map(oneexp,expo)

[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.710000000000008, 3.0] [0.3091666666666673, 0.25]
nStar =  7395
[3.7100000

In [ ]:
lcmap =v.map(oneexp,expo)
llc = lcmap.get()

In [ ]:
#lcmap3[0]

In [13]:
def mergelc(lc1,lc2):
    i=0
    for lc in lc2 :
        lc1[i].extend(lc)
        i = i+1
    return lc1
        

lcc1 = [[1,2,5],[],[1,1]]
lcc2= [[8],[8,9],[7,7,9]]
lcc3= [[3],[],[6,8]]

tll = [lcc1,lcc2,lcc3]
lc = reduce(mergelc,llc)
#lccc = mergelc(lcc1,lcc2)

In [14]:
# Calculate the I coeffs
Icoeff = [[] for z in xrange(nStar)]
wmagAduDiag = []
nObs = []
Isum = np.zeros(nStar, dtype=float)
for istar in range(nStar):
    nObs.append(len(lc[istar]))
    for iexp in range(nObs[istar]):
        Isum[istar] = Isum[istar] + lc[istar][iexp].wmagAdu
#to save all the weights in a 1D list, and create the diagonal matrix
        wmagAduDiag.append(lc[istar][iexp].wmagAdu)
    for iexp in range(nObs[istar]):
        Icoeff[istar].append(lc[istar][iexp].wmagAdu/Isum[istar])

In [15]:
print datetime.datetime.now()
print np.sum(nObs)
print len(wmagAduDiag)
print nStar


2016-10-13 17:47:20.930704
50870
50870
7395


In [16]:
# Fill the A matrix ! It's a nTotObs x nPar matrix (here nTotObs = np.sum(nObs) and nPar =  8 (k) + 2 (a) = 10)
nA = len(a); nK = ik[len(ik) - 1] + 1; nPar = nA + nK # 2 a, and 8 k    
print nA, nK
nTotObs = np.sum(nObs)
#A = [[] for z in xrange(nTotObs)]
A = np.zeros((nTotObs, nPar))
Aline = 0
for istar in range(nStar):
    nExp = nObs[istar]
# to define Aline0, as the first A line which corresponds to the istar observation
    Aline0 = Aline
    for iexp in range(nExp):
# Here I recover the night and exposure indices needed for the calculation of the columns
        night = lc[istar][iexp].night
# same a parameter for consecutive nights
        jA = ia[night]
# different k parameter for each night
        jK = ik[night]
        ixm = lc[istar][iexp].ixm
# fill the a dependent columns
        Acol = jA
        A[Aline][Acol] = A[Aline][Acol] + 1. 
        for jExp in range(nExp) :
            A[Aline0 + jExp][Acol] = A[Aline0 + jExp][Acol] - Icoeff[istar][iexp]
# fill the k dependent columns
        Acol = nA + jK
        A[Aline][Acol] = A[Aline][Acol] - x[ixm] # + 1.*(-x[ixm])
        for jExp in range(nExp) :
            A[Aline0 + jExp][Acol] = A[Aline0 + jExp][Acol] + x[ixm]*Icoeff[istar][iexp]   # -x*-I
        Aline = Aline + 1

2 4


In [17]:
print Aline, Aline0, nExp, Acol

50870 50870 0 5


In [18]:
# Calculate the avg mAdu
mAdu = []
for istar in range(nStar):
    avg = 0.
    nExp = nObs[istar]
    for iexp in range(nExp):
        avg = avg + Icoeff[istar][iexp]*lc[istar][iexp].mag_r
    mAdu.append(avg)
print len(mAdu)
# Fill the B matrix ! It's a nObs x 1 matrix
B = []
for istar in range(nStar):
    nExp = nObs[istar]
    for iexp in range(nExp):
        B.append(mAdu[istar] - lc[istar][iexp].mag_r)
#m1,1I1,1 + m1,2I1,2 - m1,1
#m1,1I1,1 + m1,2I1,2 - m1,2
print len(B)

7395
50870


In [19]:
C = np.diagflat(wmagAduDiag)
AtCA = np.dot(np.dot(np.transpose(A),C),A)
AtCB  = np.dot(np.dot(np.transpose(A),C),B)
#print AtCA
#print AtCB
akFit = np.linalg.solve(AtCA, AtCB)
print itime, datetime.datetime.now()
print akFit
cc = []
for i in range(nTotObs):
    cc.append(A[i][0]**2*C[i][i])

2016-10-13 17:36:27.558494 2016-10-13 17:49:15.447205
[ 0.21344144  0.06352696  0.09954652  0.11955831  0.10965271  0.13965301]


In [ ]:
cc = []
for i in range(nTotObs):
    cc.append(A[i][0]**2*C[i][i])
for i in range(nPar):
    print 1./np.sqrt(AtCA[i][i]), AtCA[i][i]
small = A[0][0]**2*C[0][0]
big0 = AtCA[0][0]
big1 =  big0 + small
small1 = big1 - big0
print small, big0, big1, small1
cc.sort()
print cc[0], cc[nTotObs-2], cc[nTotObs-1]

In [ ]:
print k
print akFit[1] - akFit[0], a[1] - a[0], akFit[1] - akFit[0] - (a[1] - a[0]), np.sqrt(1./AtCA[0][0] + 1./AtCA[1][1])
for i in range(nK):
    print i, akFit[i+nA], k[i], akFit[i+nA] - k[i], 1./np.sqrt(AtCA[i+nA][i+nA])

In [ ]:
i0=0;i1=0;i2=0;i3=0
for i in range(nStar):
    if (nObs[i]<4):
        i0 = i0+1
    if(3<nObs[i]<8):
        i1 = i1+1
    if(7<nObs[i]<12):
        i2 = i2+1
    if (nObs[i]>11):
        i3 = i3+1
print i0,i1,i2,i3


In [ ]:
istar =333
print len(lc[istar]) 
for i in xrange(len(lc[istar])) :
    print lc[istar][i].mag_r, lc[istar][i].night, lc[istar][i].ixm